# Коллаборативный подход. ALS.

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

from pyspark.sql import SparkSession
sc = SparkContext
spark = SparkSession.builder.appName('rec').getOrCreate()

22/04/03 20:10:58 WARN Utils: Your hostname, alexander-HP-Pavilion-Gaming-Laptop-15-cx0xxx resolves to a loopback address: 127.0.1.1; using 100.125.6.191 instead (on interface wlo1)
22/04/03 20:10:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/03 20:10:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Для коллаборативного подхода нам необходимы только id фильмов, id пользователей и рейтинги фильмов, представленные в файлах movie.csv и rating.csv.  

Загружаем датасет, используем часть данных, так как обладаем ограниченной мощностью компьютера. 

In [2]:
ratings = pd.read_csv('./rating.csv')[:100000].drop('timestamp', axis=1)
tags = pd.read_csv('./tag.csv')[:100000].drop('timestamp', axis=1)
movies = pd.read_csv('./movie.csv')[:100000]

movies = spark.createDataFrame(movies)
ratings = spark.createDataFrame(ratings)

ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')

ratings.show()
ratings.printSchema()
ratings.describe().show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
|     1|    293|   4.0|
|     1|    296|   4.0|
|     1|    318|   4.0|
|     1|    337|   3.5|
|     1|    367|   3.5|
|     1|    541|   4.0|
|     1|    589|   3.5|
|     1|    593|   3.5|
|     1|    653|   3.0|
|     1|    919|   3.5|
+------+-------+------+
only showing top 20 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)

+-------+------------------+------------------+------------------+
|summary|            userId|           movieId|            rating|
+-------+------------------+------------------+------------------+
|  count|            100000|            100000|            100000|
|   mean

Для коллаборативного подхода будем использовать алгоритм ALS реализованный в PySpark.

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

Перемешенный датасет делим на тренировочный и тестовый наборы в соотношении 8 к 2.   

Создаем модель, ставим ограничение не отрицательности. В coldStartStrategy ставим параметр 'drop', чтобы удалить любые строки в датафрейме предсказаний, содержащие NaN значения. Используем явную обратную связь.

In [4]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

Строим сетку параметров на основе количества скрытых факторов в модели и параметре регуляризации.

In [5]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
pass

Используем rmse оценку.

In [6]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

В качестве валидации используем перекрестную проверку K-Folds, (numFolds - 1) часть для тренировки и numFolds часть для тестировки.Это приводит к менее предвзятой модели по сравнению с другими методами. Потому что это гарантирует, что каждое наблюдение из исходного набора данных может появиться в обучающем и тестовом наборах.

In [7]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

Тренируем полученные модели и выбираем лучшую.

In [8]:
model = cv.fit(train)
best_model = model.bestModel

22/04/03 20:11:29 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/03 20:11:29 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


На основании полученной лучшей модели выбираем значения гиперпараметров.

In [9]:
print("**Best Model**")
print("  Rank:", best_model._java_obj.parent().getRank())
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


Используем среднеквадратичную ошибку для оценки полученных результатов.

In [10]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print("RMSE: ", RMSE)
test_predictions.show()

RMSE:  0.8961273345685145


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   490|    148|   3.0| 1.7777098|
|   233|    471|   3.0| 3.5017989|
|   504|    471|   3.0| 3.4824753|
|   387|    471|   4.5| 3.9202704|
|   445|    471|   3.0| 3.2076337|
|   448|    471|   4.5| 3.6456368|
|   394|    471|   4.0| 3.8628635|
|   469|    471|   5.0|  2.919877|
|   702|    471|   4.5| 2.9751718|
|   147|    471|   4.5|  3.433765|
|   294|    471|   3.0| 3.2621062|
|   208|    471|   3.5| 3.3740647|
|   137|   1088|   5.0| 3.6048086|
|   133|   1088|   2.5| 2.4639032|
|   375|   1088|   4.0|  3.361151|
|   140|   1088|   3.0| 3.2340083|
|   178|   1088|   4.0| 3.5193567|
|    54|   1088|   3.0| 2.9684663|
|   586|   1088|   1.0|  2.784155|
|   609|   1088|   2.0| 1.9954188|
+------+-------+------+----------+
only showing top 20 rows



Составляем 10 рекомендаций пользователю.

In [11]:
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

Предложенные фильмы.

In [12]:
nrecommendations.join(movies, on='movieId').filter('userId = 89').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|   4798|    89|4.6315928|   Indiscreet (1958)|      Comedy|Romance|
|  48326|    89|4.6836123|All the King's Me...|               Drama|
|  84775|    89|4.5823927|    Submarino (2010)|               Drama|
| 100714|    89|4.7446623|Before Midnight (...|       Drama|Romance|
|  82934|    89|4.7276993|Most Dangerous Ma...|         Documentary|
|  71033|    89|  4.68508|Secret in Their E...|Crime|Drama|Myste...|
| 109697|    89|4.5823927|Inspector Palmu's...|Comedy|Crime|Myst...|
|   5004|    89|4.7906103|   Party, The (1968)|              Comedy|
|  63033|    89|4.5741405|    Blindness (2008)|Drama|Mystery|Rom...|
|  27803|    89|4.6105523|Sea Inside, The (...|               Drama|
+-------+------+---------+--------------------+--------------------+



Фильмы предпочитаемые пользователем.

In [13]:
ratings.join(movies, on='movieId').filter('userId = 89').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   4148|    89|   5.0|     Hannibal (2001)|     Horror|Thriller|
|   3578|    89|   5.0|    Gladiator (2000)|Action|Adventure|...|
|   2707|    89|   5.0|Arlington Road (1...|            Thriller|
|   4235|    89|   5.0|Amores Perros (Lo...|      Drama|Thriller|
|    111|    89|   5.0|  Taxi Driver (1976)|Crime|Drama|Thriller|
|   4167|    89|   5.0|   15 Minutes (2001)|            Thriller|
|    318|    89|   5.0|Shawshank Redempt...|         Crime|Drama|
|    715|    89|   5.0|Horseman on the R...|       Drama|Romance|
|   2324|    89|   5.0|Life Is Beautiful...|Comedy|Drama|Roma...|
|   1127|    89|   5.0|   Abyss, The (1989)|Action|Adventure|...|
+-------+------+------+--------------------+--------------------+



# Коллаборативный + контентный подход. LightFM.

В качестве метрик будем использовать название фильма, жанры фильма и тэги фильма. Эти метрики должны оказывать наибольшее влияние на выбор пользователя.

In [14]:
import numpy as np
import pandas as pd 

ratings2 = pd.read_csv('./rating.csv')[:100000].drop('timestamp', axis=1)
tags2 = pd.read_csv('./tag.csv')[:100000].drop('timestamp', axis=1)
movies2 = pd.read_csv('./movie.csv')[:100000]

### Метаданные фильмов

Формируем средние значения рейтинга фильма. Объединяем тэги фильма, пустые тэги заменяем значением 'No tag'. Собираем все в один датафрейм. Так как используем не весь датасет, берем данные только по фильмам, которые загружены из файла movie.csv (не учитываем тэги к фильмам, которые не загрузили).

In [15]:
mean_ratings = ratings2.drop('userId', axis=1).groupby('movieId').agg('mean')
mean_ratings.rename(columns = {'rating' : 'average_rating'}, inplace = True)

In [16]:
movie_tags = tags2.groupby("movieId")['tag'].agg(lambda x: set(x)).reset_index()
movie_tags['metadata'] = movie_tags['tag'].str.join(", ")

items = pd.merge(movies2,movie_tags.drop(columns = 'tag'),how = 'left')
items['metadata']= items['metadata'].fillna('No tag')

movies_metadata_selected = pd.merge(items,mean_ratings,how = 'inner', left_on='movieId', right_on='movieId')

Замениям отсутствующее значение другим значением. Преобразуем значения интервала для числовых переменных в дискретные интервалы. 

In [17]:
movies_metadata_selected.replace('', np.nan, inplace=True)
movies_metadata_selected['average_rating'].replace(np.nan, 0, inplace=True)
movies_metadata_selected['average_rating'] = movies_metadata_selected['average_rating'].apply(lambda x: round(x*10)/10)
movies_metadata_selected = movies_metadata_selected.sort_values(by=['average_rating'])
movies_metadata_selected.sample(5)

,movieId,title,genres,metadata,average_rating
6323,42738,Underworld: Evolution (2006),Action|Fantasy|Horror,"dark fantasy, Watched, werewolf, strong-minded...",3.4
7784,88237,Griff the Invisible (2011),Comedy|Drama|Romance,No tag,3.5
2574,3125,"End of the Affair, The (1999)",Drama,Graham Greene,4.0
7850,91094,"Muppets, The (2011)",Children|Comedy|Musical,"weak plot, comeback, cameos, musical, Amy Adam...",3.3
5791,26502,"Christmas Carol, A (1984)",Drama|Fantasy,"adapted from:play, Bechdel Test:Fail, Xmas the...",2.0


### Данные о взаимодействиях

Данные соддержат id пользователя, id фильма и оценку пользователя к фильму.

In [18]:
interactions_selected = pd.merge(movies_metadata_selected,
                                ratings2,
                                how = 'inner', 
                                left_on='movieId',
                                right_on='movieId')
interactions_selected = interactions_selected.sort_values(by=['average_rating'])
interactions_selected.sample(5)

,movieId,title,genres,metadata,average_rating,userId,rating
10349,2054,"Honey, I Shrunk the Kids (1989)",Adventure|Children|Comedy|Fantasy|Sci-Fi,"Joe Johnston, Rick Moranis, kid flick, honest,...",2.8,196,3.0
93938,1192,Paris Is Burning (1990),Documentary,"queer, sad, lgbt, documentary, transgender, pr...",4.2,53,5.0
1653,181,Mighty Morphin Power Rangers: The Movie (1995),Action|Children,"Nostalgic, Power Rangers",2.0,686,1.0
27627,1441,Benny & Joon (1993),Comedy|Romance,"sweet, romantic comedy, drama, Quirky, mental ...",3.3,377,5.0
53738,2871,Deliverance (1972),Adventure|Drama|Thriller,"river, Burt Reynolds, rednecks, old movie, ban...",3.6,347,4.0


Создаем словарь для будущих ссылок. Преобразуем метаданные о фильмах в разряженную матрицу CSR.

In [19]:
from scipy.sparse import csr_matrix

item_dict ={}

movie_id_title = movies_metadata_selected[['movieId', 
    'title']].sort_values('movieId').reset_index()

for i in range(movie_id_title.shape[0]):
    item_dict[(movie_id_title.loc[i,'movieId'])] = movie_id_title.loc[i,'title']
    
movies_metadata_selected_transformed = pd.get_dummies(movies_metadata_selected)

movies_metadata_selected_transformed = \
movies_metadata_selected_transformed.sort_values('movieId').reset_index().drop('index', axis=1)


movies_metadata_csr = \
csr_matrix(movies_metadata_selected_transformed.drop('movieId',axis=1).values.T)

Создаем словарь пользователей для будущих ссылок. Создаем матрицу взаимодействий.

In [20]:
user_movie_interaction = pd.pivot_table(pd.merge(interactions_selected,
                                                 movies_metadata_selected,
                                                 how = 'inner', 
                                                 left_on='movieId',
                                                 right_on='movieId'),
        index='userId', columns='movieId', values='rating')

user_movie_interaction = user_movie_interaction.fillna(0)

user_id = list(user_movie_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1
    
user_movie_interaction_csr = csr_matrix(user_movie_interaction.values)

Разбиение данных на обучение и валидацию 0.8 к 0.2 соответственно.

In [21]:
from lightfm import cross_validation
train, test = cross_validation.\
random_train_test_split(user_movie_interaction_csr, test_percentage=0.2)

Обучаем модель.  
В качестве функции потерь используем warp, так как он показал лучшие результаты в ходе испытаний.  
Используем алгоритм оптимизации adadelta, котрый показал лучший резутьтат чем adagrad.
Входе исследования были также выбраны значения: max_sampled, rho, user_alpha и no_components.

In [22]:
from lightfm import LightFM

model = LightFM(loss='warp',
                learning_schedule = 'adadelta',
                random_state = 2022,
                no_components = 15,
                user_alpha = 0.0001,
                rho = 0.5,
                max_sampled = 5)

model = model.fit(train,
                  epochs=100,
                  item_features=movies_metadata_csr,
                  num_threads=16, verbose=False)

Выводим топ n рекомендаций.

In [23]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x, np.arange(n_items), item_features = movies_metadata_csr))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1
        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
            
sample_recommendation_user(model, user_movie_interaction, 89, user_dict, item_dict)

User: 89
Known Likes:
1- Final Fantasy: The Spirits Within (2001)
2- Cats & Dogs (2001)
3- Baby Boy (2001)
4- Lara Croft: Tomb Raider (2001)
5- Swordfish (2001)
6- Animal, The (2001)
7- Shrek (2001)
8- Angel Eyes (2001)
9- Knight's Tale, A (2001)
10- Mummy Returns, The (2001)
11- Town & Country (2001)
12- One Night at McCool's (2001)
13- Driven (2001)
14- Joe Dirt (2001)
15- Bridget Jones's Diary (2001)
16- Just Visiting (2001)
17- Blow (2001)
18- Along Came a Spider (2001)
19- Amores Perros (Love's a Bitch) (2000)
20- Tailor of Panama, The (2001)
21- Memento (2000)
22- Enemy at the Gates (2001)
23- Revenge of the Nerds II: Nerds in Paradise (1987)
24- Revenge of the Nerds (1984)
25- Harley Davidson and the Marlboro Man (1991)
26- Double Impact (1991)
27- 15 Minutes (2001)
28- Series 7: The Contenders (2001)
29- Company Man (2000)
30- Hannibal (2001)
31- Mannequin (1987)
32- Legend of Drunken Master, The (Jui kuen II) (1994)
33- Bedazzled (2000)
34- Lost Souls (2000)
35- Contender, The

Оценка полученной модели.

In [24]:
from lightfm.evaluation import auc_score

train_auc = auc_score(model, train, item_features=movies_metadata_csr).mean()
test_auc = auc_score(model, test, item_features=movies_metadata_csr, train_interactions=train).mean()

print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.96, test 0.92.


ROC AUC на тестовом наборе показывает значение 0.9, что является неплохим значением (идиально - 1). Можем сделать вывод, что составленный рейтинг имеет неплохое качество.

# Выводы
В ходе работы были реализованы 2 рекомендательные системы: ALS и LightFM. 

ALS использует только рейтинги фильмов указанные пользователями.

LightFM использует рейтинги фильмов, а также указанные тэги название и жанры. 

Используя RMSE для оценки результатов ALS, можно сказать о не очень высокой точности предсказания оценки пользователя.

Для оценки LightFM использовался AUC, на основе его значения, можно говорить более высокой точности рекомендательной системы.